In [1]:
import numpy as np
import pandas as pd

import sys
import os
from tqdm import tqdm

sys.path.append("/kaggle")
from src.utils.post_process import post_process_for_seg
from src.utils.metrics import event_detection_ap

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
exp_id = "exp012"

dfs = []
for fold in tqdm(range(5)):
    keys = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/keys.npy")
    labels = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/labels.npy")
    preds = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/preds.npy")

    sub = post_process_for_seg(
        keys=keys,
        preds=preds[:, :, [1, 2]],
        score_th=0.02,
        distance=10,
        penguin_pp=True,
    )
    dfs.append(sub)

100%|██████████| 5/5 [04:32<00:00, 54.48s/it]


In [3]:
sub = pd.concat(dfs).reset_index(drop=True)
sub.head()

,row_id,series_id,step,event,score
0,0,05e1944c3818,263141,onset,7.695312
1,1,05e1944c3818,314252,onset,7.648438
2,2,05e1944c3818,192929,onset,7.480469
3,3,05e1944c3818,210935,onset,7.468750
4,4,05e1944c3818,349341,onset,7.453125


In [4]:
labels = pd.read_csv("/kaggle/data/train_events.csv").dropna()

In [6]:
score = event_detection_ap(labels, sub)
score

0.7617573820770355